In [1]:
from dotenv import load_dotenv
import os
_ = load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

# 3. Runnable 프로토콜
LangChain의 "Runnable" 프로토콜은 사용자가 **사용자 정의 체인**을 쉽게 생성하고 관리할 수 있도록 설계된 핵심적인 개념입니다.\
일관된 인터페이스를 사용하여 다양한 타입의 컴포넌트를 조합하고, 복잡한 데이터 처리 파이프라인을 구성할 수 있습니다.

### 주요 메소드

- **invoke**: 주어진 입력에 대해 체인을 호출하고, 결과를 반환합니다. 단일 입력에 대해 동기적으로 작동합니다.

- **batch**: 입력 리스트에 대해 체인을 호출하고, 각 입력에 대한 결과를 리스트로 반환합니다. 여러 입력에 대해 동기적으로 작동하며, 효율적인 배치 처리를 가능하게 합니다.

- **stream**: 입력에 대해 체인을 호출하고, 결과의 조각들을 스트리밍합니다. 이는 대용량 데이터 처리나 실시간 데이터 처리에 유용합니다.

- **비동기 버전**: `ainvoke`, `abatch`, `astream` 등의 메소드는 각각의 동기 버전에 대한 비동기 실행을 지원합니다. 더 높은 처리 성능과 효율을 달성할 수 있습니다.


### 커스텀 체인을 생성하는 과정

1. 필요한 컴포넌트를 정의하고, 각각 "Runnable" 인터페이스를 구현합니다.

2. 컴포넌트들을 조합하여 사용자 정의 체인을 생성합니다.

3. 생성된 체인을 사용하여 데이터 처리 작업을 수행합니다. 이때, invoke, batch, stream 메소드를 사용하여 원하는 방식으로 데이터를 처리할 수 있습니다.

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser

# 1. 컴포넌트 정의
prompt = ChatPromptTemplate.from_template("지구과학에서 {topic}에 대해 간단히 설명해주세요.")
model = init_chat_model(model="gpt-4o-mini", api_key=api_key)
output_parser = StrOutputParser()

# 2. 체인 생성
chain = prompt | model | output_parser

# 3. invoke 메소드 사용
result = chain.invoke({"topic": "지구 자전"})
print("invoke 결과:", result)

invoke 결과: 지구 자전은 지구가 자전축을 중심으로 회전하는 움직임을 말합니다. 지구는 약 24시간에 한 번 자전하며, 이로 인해 낮과 밤이 생깁니다. 자전축은 지구의 북극과 남극을 연결하는 가상의 선으로, 약 23.5도의 경사를 가지고 있어 계절의 변화에도 영향을 미칩니다. 

지구의 자전 속도는 적도에서 가장 빠르고, 극지방으로 갈수록 느려집니다. 자전으로 인해 발생하는 원심력은 지구의 형태에 영향을 주어, 지구가 약간 부풀어 오른 타원체 모양을 띠게 만듭니다. 자전은 또한 조수 현상과 같은 다양한 자연 현상에도 중요한 역할을 합니다.


**batch 메소드 사용**

In [6]:
topics = ["지구 공전", "화산 활동", "대륙 이동"]
results = chain.batch([{"topic": t} for t in topics])
for topic, result in zip(topics, results):
    print(f"{topic} 설명: {result[:50]}...") # 처음 50자만 출력

지구 공전 설명: 지구 공전은 지구가 태양 주위를 원형 또는 타원형 궤도를 따라 회전하는 과정을 말합니다. ...
화산 활동 설명: 화산 활동은 지구 내부의 마그마가 지표로 분출되는 과정을 말합니다. 이 과정은 주로 지구의...
대륙 이동 설명: 대륙 이동 이론은 지구의 표면에 있는 대륙들이 시간이 지남에 따라 이동한다는 것을 설명하는...


**stream 메소드 사용**

In [9]:
stream = chain.stream({"topic": "지진"})
print("stream 결과:")
for chunk in stream:
    print(chunk, end='', flush=True)

stream 결과:
지진은 지구 내부의 지각 또는 맨틀에서 발생하는 에너지 방출로 인해 발생하는 자연재해입니다. 이 에너지는 보통 지각의 단층이나 암석의 변형에 의해 발생하며, 이러한 에너지가 갑작스럽게 방출될 때 지표면에서 진동이 느껴지게 됩니다. 

지진은 주로 판의 경계에서 발생하며, 세 가지 주요 유형의 경계가 있습니다: 

1. **발산 경계**: 두 판이 서로 멀어지면서 새로운 지각이 형성되는 곳.
2. **수축 경계**: 두 판이 서로 가까워지며 하나의 판이 다른 판 아래로 끌려 들어가는 곳.
3. **변동 경계**: 두 판이 서로 미끄러지면서 발생하는 곳.

지진의 강도는 리히터 규모나 모멘트 규모로 측정되며, 강한 지진일수록 피해가 클 수 있습니다. 지진이 발생하면 지반이 흔들리면서 건물이나 인프라에 피해를 줄 수 있으며, 경우에 따라 쓰나미와 같은 2차 재해가 발생할 수 있습니다. 

지진을 예측하는 것은 매우 어려운 문제이며, 각종 센서와 데이터 분석을 통해 발생 가능성을 연구하고 있습니다.

**ainvoke 메소드 사용**

In [10]:
import nest_asyncio
import asyncio

# nest_asyncio 적용 (구글 코랩 등 주피터 노트북에서 실행 필요)
nest_asyncio.apply()

# 비동기 메소드 사용 (async/await 구문 필요)
async def run_async():
    result = await chain.ainvoke({"topic": "해류"})
    print("ainvoke 결과:", result[:50], "...")

asyncio.run(run_async())

ainvoke 결과: 해류는 바다에서 물이 일정한 방향으로 흐르는 현상을 말합니다. 이들은 해양의 물리적, 화학 ...
